In [2]:
import sys
sys.path.insert(0, '/app')
sys.path.insert(0, '/app/examples/strategies')
sys.path.insert(0, '/app/examples/utils')

# Register bundles FIRST
try:
    from register_bundles import ensure_bundles_registered
    ensure_bundles_registered(verbose=True)
    print("✅ Bundles registered")
except Exception as e:
    print(f"⚠️  Bundle registration failed: {e}")
    print("Trying manual registration...")
    
    # Manual registration as fallback
    from zipline.data.bundles import register
    from zipline.data.bundles.sharadar_bundle import sharadar_bundle
    
    register('sharadar', sharadar_bundle(
        tickers=None,
        incremental=True,
        include_funds=True,
    ))
    print("✅ Bundles manually registered")

from zipline.pipeline import Pipeline
from zipline.data.bundles import load as load_bundle
from zipline.pipeline.loaders.auto_loader import setup_auto_loader
from zipline.pipeline import SimplePipelineEngine
import pandas as pd

# Import our custom filters
from sharadar_filters import (
    ExchangeFilter,
    CategoryFilter,
    ADRFilter,
    SectorFilter,
    ScaleMarketCapFilter,
    create_sharadar_universe,
    SharadarTickers,
)

print("✅ Imports successful")

✓ Bundles already registered
✅ Bundles registered
✅ Imports successful


## 1. Set up Pipeline Engine

In [2]:
# Set up auto loader
print("Setting up auto loader...")
custom_loader = setup_auto_loader(
    bundle_name='sharadar',
    custom_db_dir='/data/custom_databases',
    enable_sid_translation=True,
)

# Load bundle
print("Loading bundle...")
bundle_data = load_bundle('sharadar')

# Create pipeline engine
print("Creating pipeline engine...")
engine = SimplePipelineEngine(
    get_loader=lambda column: custom_loader,
    asset_finder=bundle_data.asset_finder,
    default_domain=bundle_data.equity_daily_bar_reader.trading_calendar.day,
)

print("✅ Pipeline engine ready")

ERROR! Session/line number was not unique in database. History logging moved to new session 89
Setting up auto loader...
Loading bundle...
Creating pipeline engine...
Unexpected exception formatting exception. Falling back to standard exception
Unexpected exception formatting exception. Falling back to standard exception
Unexpected exception formatting exception. Falling back to standard exception


# Set up auto loader
print("Setting up auto loader...")
custom_loader = setup_auto_loader(
    bundle_name='sharadar',
    custom_db_dir=None,
    enable_sid_translation=True,
)

# Load bundle
print("Loading bundle...")
bundle_data = load_bundle('sharadar')

# Create pipeline engine
print("Creating pipeline engine...")

# Get the trading calendar from the bundle
from zipline.utils.calendar_utils import get_calendar
trading_calendar = get_calendar('XNYS')

try:
    engine = SimplePipelineEngine(
        get_loader=lambda column: custom_loader,
        asset_finder=bundle_data.asset_finder,
        default_domain=trading_calendar.day,
    )
    print("✅ Pipeline engine ready")
except Exception as e:
    print(f"Error creating pipeline engine: {e}")
    import traceback
    traceback.print_exc()
    raise

In [ ]:
# Test pipeline with individual filtersdef make_test_pipeline():    # Get raw metadata    exchange = SharadarTickers.sharadar_exchange.latest    category = SharadarTickers.sharadar_category.latest    is_adr = SharadarTickers.sharadar_is_adr.latest    sector = SharadarTickers.sharadar_sector.latest    scalemarketcap = SharadarTickers.sharadar_scalemarketcap.latest        # Create filters    exchange_filter = ExchangeFilter()    category_filter = CategoryFilter()    adr_filter = ADRFilter()        return Pipeline(        columns={            'exchange': exchange,            'category': category,            'is_adr': is_adr,            'sector': sector,            'scalemarketcap': scalemarketcap,            'exchange_filter': exchange_filter,            'category_filter': category_filter,            'adr_filter': adr_filter,        },    )# Run for one datetest_date = pd.Timestamp('2024-01-05')print(f"Running pipeline for {test_date}...")pipeline = make_test_pipeline()result = engine.run_pipeline(pipeline, test_date, test_date)print(f"\n✅ Pipeline executed: {len(result)} assets")print(f"\nSample data:")print(result.head(20))

## 3. Check Filter Results

In [ ]:
# Check how many assets pass each filter
print("=" * 80)
print("FILTER RESULTS")
print("=" * 80)

total_assets = len(result)
print(f"Total assets: {total_assets:,}")
print()

# Exchange filter
exchange_pass = result['exchange_filter'].sum()
print(f"Exchange Filter (NYSE/NASDAQ/NYSEMKT): {exchange_pass:,} ({exchange_pass/total_assets*100:.1f}%)")

# Category filter
category_pass = result['category_filter'].sum()
print(f"Category Filter (Domestic Common Stock): {category_pass:,} ({category_pass/total_assets*100:.1f}%)")

# ADR filter (True means IS an ADR, so we want to exclude these)
adr_count = result['adr_filter'].sum()
non_adr_count = total_assets - adr_count
print(f"ADR count: {adr_count:,} ({adr_count/total_assets*100:.1f}%)")
print(f"Non-ADR count: {non_adr_count:,} ({non_adr_count/total_assets*100:.1f}%)")

# Combined filter
combined_filter = result['exchange_filter'] & result['category_filter'] & ~result['adr_filter']
combined_pass = combined_filter.sum()
print(f"\nCombined Filter (all three): {combined_pass:,} ({combined_pass/total_assets*100:.1f}%)")

print()
print("=" * 80)

## 4. View Distribution

In [ ]:
# Exchange distribution
print("\n📊 Exchange Distribution:")
print(result['exchange'].value_counts())

# Category distribution
print("\n📊 Category Distribution (top 10):")
print(result['category'].value_counts().head(10))

# Sector distribution (for filtered stocks)
filtered_stocks = result[combined_filter]
print(f"\n📊 Sector Distribution (filtered {len(filtered_stocks):,} stocks):")
print(filtered_stocks['sector'].value_counts())

# Market cap scale distribution (for filtered stocks)
print(f"\n📊 Market Cap Scale (filtered stocks):")
print(filtered_stocks['scalemarketcap'].value_counts())

## 5. Test Combined Universe Function

In [ ]:
# Test the create_sharadar_universe helper functiondef make_universe_test_pipeline():    # Create standard universe    sharadar_universe = create_sharadar_universe(        exchanges=['NYSE', 'NASDAQ', 'NYSEMKT'],        include_adrs=False,    )        return Pipeline(        columns={            'exchange': SharadarTickers.sharadar_exchange.latest,            'category': SharadarTickers.sharadar_category.latest,        },        screen=sharadar_universe,    )# Run pipeline with screenuniverse_pipeline = make_universe_test_pipeline()universe_result = engine.run_pipeline(universe_pipeline, test_date, test_date)print(f"\n✅ Universe Filter Applied: {len(universe_result):,} stocks pass")print(f"\nExchange distribution in filtered universe:")print(universe_result['exchange'].value_counts())print(f"\nCategory distribution in filtered universe:")print(universe_result['category'].value_counts())

## 6. Test Advanced Filters

In [ ]:
# Test sector filterdef make_sector_test_pipeline():    tech_universe = create_sharadar_universe(        sectors=['Technology'],        include_adrs=False,    )        return Pipeline(        columns={            'sector': SharadarTickers.sharadar_sector.latest,            'industry': SharadarTickers.sharadar_industry.latest,        },        screen=tech_universe,    )tech_pipeline = make_sector_test_pipeline()tech_result = engine.run_pipeline(tech_pipeline, test_date, test_date)print(f"\n✅ Technology Sector Filter: {len(tech_result):,} stocks")print(f"\nIndustry distribution in Tech sector:")print(tech_result['industry'].value_counts())

## 7. Test Market Cap Scale Filter

In [ ]:
# Test large-cap filterdef make_largecap_test_pipeline():    large_cap_universe = create_sharadar_universe(        include_adrs=False,        min_market_cap_scale=5,  # Large (5) and Mega (6) cap    )        return Pipeline(        columns={            'scalemarketcap': SharadarTickers.sharadar_scalemarketcap.latest,            'sector': SharadarTickers.sharadar_sector.latest,        },        screen=large_cap_universe,    )largecap_pipeline = make_largecap_test_pipeline()largecap_result = engine.run_pipeline(largecap_pipeline, test_date, test_date)print(f"\n✅ Large-Cap Filter (scale 5-6): {len(largecap_result):,} stocks")print(f"\nMarket cap scale distribution:")print(largecap_result['scalemarketcap'].value_counts())print(f"\nSector distribution in large-caps:")print(largecap_result['sector'].value_counts())

## Summary

If all cells above executed successfully, the Sharadar ticker metadata filters are working correctly!

You can now use these filters in your strategies like this:

```python
from sharadar_filters import create_sharadar_universe

def make_pipeline():
    # Create base universe
    universe = create_sharadar_universe(
        exchanges=['NYSE', 'NASDAQ', 'NYSEMKT'],
        include_adrs=False,
    )
    
    # Apply additional filters
    market_cap = CustomFundamentals.CompanyMarketCap.latest
    top_1500 = market_cap.top(1500, mask=universe)
    
    # ... rest of pipeline logic
```